In [3]:
#######################
# Author: Thomas Calvet
# Instructor: Naci Dilekli 
# Geospatial programming
# (http://www.rudn.ru/ - https://www.epf.fr/)
#######################


# Initialisation of the function and its parameters

def CityChoice(workspace, USA_NPP, NPP_D, USA_NTS, NTS_D, USA_GPP, GPP_D, USA_CPP, CPP_D, Population, Vacancy, Field, Method):

# Estimate the duration execution of the function

    from datetime import datetime
    start_time = datetime.now()
    
# Getting the workspace ready, importing the library, setting things up    
    print("Preparing the function to run...")
    import arcpy
    arcpy.env.overwriteOutput = True
    arcpy.env.workspace = "C:/Thomas_Calvet_City_Choice/City_Choice_In_Python/City_Choice_In_Python.gdb"
    workspace = arcpy.env.workspace
    print("Setting up the workspace", workspace)
    
# Creating a list of all temporary layers
    
    intermediate_layers = ["USA_NPP_Buffer","USA_NTS_Buffer","USA_Nuclear_Risk","USA_GPP_Buffer","USA_CPP_Buffer",
                           "USA_Human_Activities_Air_Pollution","USA_Total_Risks_Areas","Less_Polluted_Cities", 
                           "Les_Polluted_Cities_Population", "Less_Polluted_Cities_Population_Vacancy",
                           "Less_Polluted_Cities_Population_Vacancy_Ranked"]

# Doing a for loop to delete all layers each time the function is run and thus prevent errors or false results
    
    print("Cleaning all residual layers, please wait...")
    for intermediate_layer in intermediate_layers:
        if arcpy.Exists(intermediate_layer):
            arcpy.Delete_management(intermediate_layer)
    print("\nStarting function... Function is running")
    
# Creating buffers around Nuclear Power Plants to set up "high nuclear risks areas"

    print("\nCreating nuclear power plants buffers of", str(NPP_D), "...")
    arcpy.Buffer_analysis("C:/Thomas_Calvet_City_Choice/City_Choice_In_Python/data/USA_NPP.shp", 
                          "USA_NPP_Buffer", NPP_D,"","", "ALL","")
    print("Nuclear power plants buffers created.")

# Initializing buffers around Nuclear Tests Sites

    print("Creating nuclear test sites buffers of", str(NTS_D), "...")
    arcpy.Buffer_analysis("C:/Thomas_Calvet_City_Choice/City_Choice_In_Python/data/USA_NTS.shp", 
                          "USA_NTS_Buffer", NTS_D,"","", "ALL","")
    print("Test sites buffers created.")
    
# Union of buffers around Nuclear Tests Sites & Nuclear Power Plants (Nuclear Risks)

    print("Unionizing nuclear risks areas...")
    inFeatures = [["USA_NPP_Buffer"], ["USA_NTS_Buffer"]]
    arcpy.Union_analysis(inFeatures, "USA_Nuclear_Risk")
    print("\nHigh nuclear risks layer created.")
    
# Buffers around Gas and Coal Power Plants

    print("\nCreating gas power plants buffers of", str(GPP_D), "...")
    arcpy.Buffer_analysis("C:/Thomas_Calvet_City_Choice/City_Choice_In_Python/data/USA_GPP.shp", 
                          "USA_GPP_Buffer", GPP_D,"","", "ALL","")
    print("Gas power plants buffers created.")
    
    print("Creating coal power plants buffers of", str(CPP_D), "...")
    arcpy.Buffer_analysis("C:/Thomas_Calvet_City_Choice/City_Choice_In_Python/data/USA_CPP.shp", 
                          "USA_CPP_Buffer", CPP_D,"","", "ALL","")
    print("Coal power plants buffers created.")
    
# Union of buffers around Gas & Coal Power Plants (Human Risks)

    print("Unionizing human pollution layers...")
    inFeatures = [["USA_CPP_Buffer"], ["USA_GPP_Buffer"]]
    arcpy.Union_analysis(inFeatures, "USA_Human_Activities_Air_Pollution")
    print("\nHuman pollution layer created.")
    
# Union of all buffers (Total Risk USA)

    print("\nCreating general risk layer")
    inFeatures = [["USA_Nuclear_Risk"], ["USA_Human_Activities_Air_Pollution"]]
    arcpy.Union_analysis(inFeatures, "USA_Total_Risks_Areas")
    print("General risks layer generated.")
    
# Remove all the cities that are into USA_Total_Risks_Areas
    
    lyrfile = "C:/Thomas_Calvet_City_Choice/City_Choice_In_Python/Major_Cities_US/USA_Major_Cities.shp"
    result = arcpy.GetCount_management(lyrfile)
    print('\n {} cities are considered as input'.format(result[0]))
    inFeatures = "C:/Thomas_Calvet_City_Choice/City_Choice_In_Python/Major_Cities_US/USA_Major_Cities.shp"
    erase_features = "USA_Total_Risks_Areas"
    arcpy.Erase_analysis(inFeatures, erase_features, "Less_Polluted_Cities")
    lyrfile2 = "Less_Polluted_Cities"
    result2 = arcpy.GetCount_management(lyrfile2)
    print('\nThe number of cities after substraction is {}'.format(result2[0]))
    
# Select only cities that are outside the risks areas 
# AND have a population count higher of a user entered parameter

    print("\nSelection of cities with a minimum of", str(Population), "people.")
    inFeatures = "Less_Polluted_Cities"
    outLocation = "C:/Thomas_Calvet_City_Choice/City_Choice_In_Python/City_Choice_In_Python.gdb"
    outFeatureClass = "Less_Polluted_Cities_Population"
    delimitedField = arcpy.AddFieldDelimiters("C:/Thomas_Calvet_City_Choice/City_Choice_In_Python/Major_Cities_US/USA_Major_Cities.shp",
                                              "POPULATION")
    expression = "POPULATION >" + str(Population)
    arcpy.FeatureClassToFeatureClass_conversion(inFeatures, outLocation, outFeatureClass, expression)
    print("Cities with more than", str(Population), "selected.")
    lyrfile3 = "Less_Polluted_Cities_Population"
    result3 = arcpy.GetCount_management(lyrfile3)
    print("\nThe number of cities that contains more than", str(Population), 'people is {}'.format(result3[0]))
    
# Select only cities that are outside the risks areas 
# AND have a population count higher of a user entered parameter
# AND have a vacancy rate higher than a user entered parameter

    print("\nLooking for cities with more than", str(Vacancy), "vacant houses.")
    inFeatures = "Less_Polluted_Cities_Population"
    outLocation = "C:/Thomas_Calvet_City_Choice/City_Choice_In_Python/City_Choice_In_Python.gdb"
    outFeatureClass = "Less_Polluted_Cities_Population_Vacancy"
    delimitedField = arcpy.AddFieldDelimiters("C:/Thomas_Calvet_City_Choice/City_Choice_In_Python/Major_Cities_US/USA_Major_Cities.shp",
                                              "VACANT")
    expression = "VACANT >" + str(Vacancy)
    arcpy.FeatureClassToFeatureClass_conversion(inFeatures, outLocation, outFeatureClass, expression)
    print("Selection of cities with sufficient vacancy rate.")
    lyrfile4 = "Less_Polluted_Cities_Population_Vacancy"
    result4 = arcpy.GetCount_management(lyrfile4)
    print("\nThe number of cities that contains more than", str(Vacancy), 'empty houses is {}'.format(result4[0]))
    
# Sorting all cities from the final table according to their population

    print("\nSorting all results by", str(Field), "with the following method:", str(Method))
    in_dataset = "Less_Polluted_Cities_Population_Vacancy"
    out_dataset = "Less_Polluted_Cities_Population_Vacancy_Ranked"
    sort_field = [[str(Field), str(Method)]]
    sort_method = "PEANO" # space filling curve algorithm
    arcpy.Sort_management(in_dataset, out_dataset, sort_field, sort_method)
    lyrfile5 = "Less_Polluted_Cities_Population_Vacancy_Ranked"
    result5 = arcpy.GetCount_management(lyrfile5)
    print("The cities layer fulfilling all requirements have been created in ArcGIS.")
    print('\nThe number of cities fulfilling all requirements: {}'.format(result5[0]))


    
# Printing the best city
    
    in_table = "Less_Polluted_Cities_Population_Vacancy_Ranked"
    fields = ["*"]
    index=1
    with arcpy.da.SearchCursor(in_table, fields) as cursor:
            for row in cursor:
                print(index,"-",row[3],",",row[5])
                index=index+1
    
# Ending the execution duration

    end_time = datetime.now()
    print('\nExecution time: {}'.format(end_time - start_time))

In [8]:
CityChoice("C:/Thomas_Calvet_City_Choice/City_Choice_In_Python/City_Choice_In_Python.gdb",
           "USA_NPP", "30 miles",
           "USA_NTS", "100 miles",
           "USA_GPP", "25 miles", 
           "USA_CPP", "25 miles",
           80000, 5000,
          "POPULATION", "DESCENDING")

Preparing the function to run...
Setting up the workspace C:/Thomas_Calvet_City_Choice/City_Choice_In_Python/City_Choice_In_Python.gdb
Cleaning all residual layers, please wait...

Starting function... Function is running

Creating nuclear power plants buffers of 30 miles ...
Nuclear power plants buffers created.
Creating nuclear test sites buffers of 100 miles ...
Test sites buffers created.
Unionizing nuclear risks areas...

High nuclear risks layer created.

Creating gas power plants buffers of 25 miles ...
Gas power plants buffers created.
Creating coal power plants buffers of 25 miles ...
Coal power plants buffers created.
Unionizing human pollution layers...

Human pollution layer created.

Creating general risk layer
General risks layer generated.

 3886 cities are considered as input

The number of cities after substraction is 638

Selection of cities with a minimum of 80000 people.
Cities with more than 80000 selected.

The number of cities that contains more than 80000 people